In [ ]:
#%pip install pybamm -q    # install PyBaMM if it is not installed
import os
import matplotlib.pyplot as plt
import numpy as np
import pybamm
import pandas as pd
import timeit
from matplotlib import style
style.use('ggplot')
start = timeit.default_timer()
os.chdir(pybamm.__path__[0]+'/..')
pybamm.set_logging_level("INFO")

In [ ]:
model = pybamm.lithium_ion.BasicDFNComposite()
chemistry = pybamm.parameter_sets.Chen2020_composite
param = pybamm.ParameterValues(chemistry=chemistry)
# model = pybamm.lithium_ion.DFN()
# param = model.default_parameter_values
param.update({"Upper voltage cut-off [V]": 4.5})
param.update({"Lower voltage cut-off [V]": 2})
param.update({
    "Maximum concentration in negative electrode of phase 1 [mol.m-3]":28700,
    "Initial concentration in negative electrode of phase 1 [mol.m-3]":23108,
    "Negative electrode mass fraction of phase 1":0.9,
    "Negative electrode diffusivity of phase 1 [m2.s-1]":3.3E-14,
    "Negative electrode mass fraction of phase 2":0.1,
    "Initial concentration in negative electrode of phase 2 [mol.m-3]":264100,
    })

C_rates = 0.01
period = 3600 / C_rates
rest_period = 3600
transition = 10
t1 = np.linspace(0, rest_period - transition, 201)
t2 = np.linspace(rest_period - transition, rest_period + transition, 51)
t3 = np.linspace(rest_period + transition, rest_period + period - transition, 401)
t4 = np.linspace(rest_period + period - transition, rest_period + period + transition, 51)
t5 = np.linspace(rest_period + period + transition, rest_period + period * 2, 401)
t_evals = np.concatenate((t1,t2,t3,t4,t5), axis=None)
t_evals = np.unique(t_evals)
capacity = param["Nominal cell capacity [A.h]"]
current = C_rates * capacity


def I_fun(A):
    def current(t):
        C_rates = 0.01
        period = 3600 / C_rates
        rest_period = 3600
        k = 10
        rest = pybamm.sigmoid( t , rest_period, k)
        discharge = pybamm.sigmoid( t , rest_period + period, k) - rest
        charge = pybamm.sigmoid( rest_period + period, t, k)
        return A * discharge - A * charge
        # return A * (t < period) - A * (t >= period)
        # return A * 0.2 * pybamm.sin(2 * np.pi * t / 7200 )
    return current


param["Current function [A]"] = I_fun(current)

sim1 = pybamm.Simulation(
    model,
    parameter_values=param,
    solver=pybamm.CasadiSolver(dt_max = 10),
)
solution = sim1.solve(t_eval = t_evals)
stop = timeit.default_timer()
print("running time: " + str(stop - start) + "s")

In [ ]:
# plot
plot = pybamm.QuickPlot( solution, [       
        "Current [A]",
        "Terminal voltage [V]",
        "X-averaged negative electrode open circuit potential of phase 1 [V]",
        "X-averaged negative electrode open circuit potential of phase 2 [V]",
    ]
)
plot.dynamic_plot()


In [ ]:
current = solution["Current [A]"].entries
times = solution["Time [s]"].entries / 3600
voltage = solution["Terminal voltage [V]"].entries
ocp_p1 = solution["X-averaged negative electrode open circuit potential of phase 1 [V]"].entries
ocp_p2 = solution["X-averaged negative electrode open circuit potential of phase 2 [V]"].entries
ocp_chen = pd.read_csv ("wip/data/negative_ocp_LGM50.txt", delimiter= '\s+',header=None,comment='#')
capacity = current * 0
capacity_max = 0
time_discharge = 0
for i in range(0,len(current)):
    if current[i]>0:
        capacity[i] = (times[i] -rest_period / 3600) * current[i]
        capacity_max = max(capacity_max, capacity[i])
        time_discharge = times[i]
    else:
        capacity[i] = capacity_max + (times[i] - time_discharge) * current[i]
capacity = 5 - capacity

In [ ]:
plt.plot(capacity,voltage)
plt.xlabel("Capacity [Ah]")
plt.ylabel("Voltage [V]")
plt.title("Cell voltage [V]")

In [ ]:
plt.plot(capacity,ocp_p1,'r--',label='Graphite')
plt.plot(capacity,ocp_p2,'b:',label='Silicon')
plt.plot(ocp_chen.values[:,0],ocp_chen.values[:,1],'k-',label='Composite (Chen et al. 2020)')
plt.xlabel("Capacity [Ah]")
plt.ylabel("Open circuit potential [V]")
plt.title("Cell open circuit potential [V]")
plt.legend()

In [ ]:
c_s_xrav_n_p1 = solution["Averaged negative electrode concentration of phase 1"].entries
c_s_xrav_n_p2 = solution["Averaged negative electrode concentration of phase 2"].entries
plt.plot(capacity, c_s_xrav_n_p1,'r-', label='Graphite')
plt.plot(capacity, c_s_xrav_n_p2, 'b:',label='Silicon')
plt.xlabel("Capacity [Ah]")
plt.ylabel("$c/c_\mathrm{max}$")
plt.ylim([0,1])
plt.legend()

In [ ]:
j_n_p1_av = solution["X-averaged negative electrode interfacial current density of phase 1 [A.m-2]"].entries
j_n_p2_av = solution["X-averaged negative electrode interfacial current density of phase 2 [A.m-2]"].entries
plt.plot(times, j_n_p1_av,'r-', label='Graphite')
plt.plot(times, j_n_p2_av, 'b:',label='Silicon')
plt.xlabel("Time [h]")
plt.ylabel("Interfacial current density [A/m$^{2}$]")
#plt.ylim([-0.05, 0.05])
plt.legend()

In [ ]:
return

In [ ]:
c_n_max= param["Maximum concentration in negative electrode [mol.m-3]"]
F = 96485.3329 #s A / mol
L_n =  param["Negative electrode thickness [m]"]
L_p =  param["Positive electrode thickness [m]"]
L_s =  param["Separator thickness [m]"]
I_typ = param["Typical current [A]"]
ne= param["Number of electrodes connected in parallel to make a cell"]
L_width = param["Electrode width [m]"]
L_height = param["Electrode height [m]"]
timescale = c_n_max * F * (L_p + L_n + L_s) / abs(I_typ / (ne * L_width * L_height))
print(timescale)

In [ ]:
print(min(c_s_xrav_n_p1))